# Основы глубинного обучения, майнор ИАД

## Домашнее задание 1. Введение в PyTorch. Полносвязные нейронные сети.

### Общая информация

Дата выдачи: 20.09.2022

Мягкий дедлайн: 23:59MSK 04.10.2022

Жесткий дедлайн: 23:59MSK 10.10.2022

### Оценивание и штрафы
Максимально допустимая оценка за работу — 10 баллов. За каждый день просрочки снимается 1 балл. Сдавать задание после жёсткого дедлайна сдачи нельзя.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.
Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

### О задании

В этом задании вам предстоит предсказывать год выпуска песни (**задача регрессии**) по некоторым звуковым признакам: [данные](https://archive.ics.uci.edu/ml/datasets/yearpredictionmsd). В ячейках ниже находится код для загрузки данных. Обратите внимание, что обучающая и тестовая выборки располагаются в одном файле, поэтому НЕ меняйте ячейку, в которой производится деление данных.

In [ ]:
!pip install wandb -qU

In [ ]:
import torch
from torch import nn
import pandas as pd
import numpy as np
import random

from tqdm.auto import tqdm
import wandb

In [ ]:
wandb.login()

In [ ]:
!wget -O data.txt.zip https://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip

In [ ]:
df = pd.read_csv('data.txt.zip', header=None)
df.head()

In [ ]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

train_size = 463715
X_train = X[:train_size, :]
y_train = y[:train_size]
X_test = X[train_size:, :]
y_test = y[train_size:]

## Задание 0. (0 баллов, но при невыполнении максимум за все задание &mdash; 0 баллов)

Мы будем использовать RMSE как метрику качества. Для самого первого бейзлайна обучите `Ridge` регрессию из `sklearn`. Кроме того, посчитайте качество при наилучшем константном прогнозе.

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error

In [ ]:
# applying srandard scaler 
standartizer = StandardScaler().fit(X_train)
X_train = standartizer.transform(X_train)
X_test = standartizer.transform(X_test)

standartizer = StandardScaler().fit(y_train.reshape(-1, 1))
y_train = standartizer.transform(y_train.reshape(-1, 1))
y_test = standartizer.transform(y_test.reshape(-1, 1))

In [ ]:
rr = Ridge(alpha = 0.05).fit(X_train, y_train)
mean_squared_error(standartizer.inverse_transform(y_test), standartizer.inverse_transform(rr.predict(X_test)), squared=False)

In [ ]:
# наилучший константный прогноз - среднее
mean_squared_error(standartizer.inverse_transform(y_test), standartizer.inverse_transform(np.full_like(y_test, np.mean(y_train))), squared=False)

## Задание 1. (максимум 10 баллов)

Реализуйте обучение и тестирование нейронной сети для предоставленного вам набора данных. Соотношение между полученным значением метрики на тестовой выборке и баллами за задание следующее:

- $\text{RMSE} \le 9.00 $ &mdash; 4 балла
- $\text{RMSE} \le 8.90 $ &mdash; 6 баллов
- $\text{RMSE} \le 8.80 $ &mdash; 8 баллов
- $\text{RMSE} \le 8.75 $ &mdash; 10 баллов

Есть несколько правил, которых вам нужно придерживаться:

- Весь пайплайн обучения должен быть написан на PyTorch. При этом вы можете пользоваться другими библиотеками (`numpy`, `sklearn` и пр.), но только для обработки данных. То есть как угодно трансформировать данные и считать метрики с помощью этих библиотек можно, а импортировать модели из `sklearn` и выбивать с их помощью требуемое качество &mdash; нельзя. Также нельзя пользоваться библиотеками, для которых сам PyTorch является зависимостью.

- Мы никак не ограничиваем ваш выбор архитектуры модели, но скорее всего вам будет достаточно полносвязной нейронной сети.

- Для обучения запрещается использовать какие-либо иные данные, кроме обучающей выборки.

- Ансамблирование моделей запрещено.

### Полезные советы:

- Очень вряд ли, что у вас с первого раза получится выбить качество на 10 баллов, поэтому пробуйте разные архитектуры, оптимизаторы и значения гиперпараметров. В идеале при запуске каждого нового эксперимента вы должны менять что-то одно, чтобы точно знать, как этот фактор влияет на качество.

- Не забудьте, что для улучшения качества модели вам поможет **нормировка таргета**.

- Тот факт, что мы занимаемся глубинным обучением, не означает, что стоит забывать про приемы, использующиеся в классическом машинном обучении. Так что обязательно проводите исследовательский анализ данных, отрисовывайте нужные графики и не забывайте про масштабирование и подбор гиперпараметров.

- Вы наверняка столкнетесь с тем, что ваша нейронная сеть будет сильно переобучаться. Для нейросетей существуют специальные методы регуляризации, например, dropout ([статья](https://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf)) и weight decay ([блогпост](https://towardsdatascience.com/weight-decay-l2-regularization-90a9e17713cd)). Они, разумеется, реализованы в PyTorch. Попробуйте поэкспериментировать с ними.

- Если вы чего-то не знаете, не гнушайтесь гуглить. В интернете очень много полезной информации, туториалов и советов по глубинному обучению в целом и по PyTorch в частности. Но не забывайте, что за скатанный код без ссылки на источник придется ответить по всей строгости!

- Если вы сразу реализуете обучение на GPU, то у вас будет больше времени на эксперименты, так как любые вычисления будут работать быстрее. Google Colab предоставляет несколько GPU-часов (обычно около 8-10) в сутки бесплатно.

- Чтобы отладить код, можете обучаться на небольшой части данных или даже на одном батче. Если лосс на обучающей выборке не падает, то что-то точно идет не так!

- Пользуйтесь утилитами, которые вам предоставляет PyTorch (например, Dataset и Dataloader). Их специально разработали для упрощения разработки пайплайна обучения.

- Скорее всего вы захотите отслеживать прогресс обучения. Для создания прогресс-баров есть удобная библиотека `tqdm`.

- Быть может, вы захотите, чтобы графики рисовались прямо во время обучения. Можете воспользоваться функцией [clear_output](http://ipython.org/ipython-doc/dev/api/generated/IPython.display.html#IPython.display.clear_output), чтобы удалять старый график и рисовать новый на его месте.

**ОБЯЗАТЕЛЬНО** рисуйте графики зависимости лосса/метрики на обучающей и тестовой выборках в зависимости от времени обучения. Если обучение занимает относительно небольшое число эпох, то лучше рисовать зависимость от номера шага обучения, если же эпох больше, то рисуйте зависимость по эпохам. Если проверяющий не увидит такого графика для вашей лучшей модели, то он в праве снизить баллы за задание.

**ВАЖНО!** Ваше решение должно быть воспроизводимым. Если это не так, то проверяющий имеет право снизить баллы за задание. Чтобы зафиксировать random seed, воспользуйтесь функцией из ячейки ниже.



In [ ]:
bs = 32
n_epochs = 10
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
best_model_path = 'best-model-parameters.pt'

In [ ]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [ ]:
set_random_seed(1337)

Вы можете придерживаться любой адекватной струкуры кода, но мы советуем воспользоваться следующими сигнатурами функций. Лучше всего, если вы проверите ваши предсказания ассертом: так вы убережете себя от разных косяков, например, что вектор предсказаний состоит из всего одного числа. В любом случае, внимательно следите за тем, для каких тензоров вы считаете метрику RMSE. При случайном или намеренном введении в заблуждение проверяющие очень сильно разозлятся.

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    """
    Our custom dataset
    """
    def __init__(self, x, y):
        self.x = torch.FloatTensor(x).to(device)
        self.y = torch.FloatTensor(y).to(device)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx, :], self.y[idx]

In [ ]:
import torch.nn.functional as F

In [ ]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self,yhat,y):
        return torch.sqrt(self.mse(yhat,y))

In [ ]:
train_set = CustomDataset(x = X_train, y = y_train)
train_loader = torch.utils.data.DataLoader(train_set,
                                           batch_size=bs,
                                           shuffle = True)

test_set = CustomDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_set,
                                          batch_size=bs,
                                          shuffle = True)

model = nn.Sequential(
    nn.Linear(X.shape[1], 100), 
    nn.ReLU(),
    nn.Linear(100, 256),
    nn.ReLU(), 
    nn.Linear(256, 64),
    nn.ReLU(), 
    nn.Linear(64, 1),
  ).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay = 0.001)
criterion = RMSELoss()

In [ ]:
def train(model, optimizer, criterion, train_loader, test_loader):
    '''
    params:
        model - torch.nn.Module to be fitted
        optimizer - model optimizer
        criterion - loss function from torch.nn
        train_loader - torch.utils.data.Dataloader with train set
        test_loader - torch.utils.data.Dataloader with test set
                      (if you wish to validate during training)
    '''
    wandb.init(project="pytorch-hw-1")
    wandb.watch(model)

    var_rmse_real_min = float("inf")

    for epoch in range(n_epochs):
        # training 
        if(epoch >= 2):
          optimizer.param_groups[0]['lr'] /= 2

        train_loss = []
        train_rmse_real = []
        for x_tr, y_tr in tqdm(train_loader):
            y_pred = model(x_tr)
            loss = criterion(y_pred, y_tr)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            train_loss.append(loss.item())

        train_rmse_real = mean_squared_error(
            standartizer.inverse_transform(model(torch.FloatTensor(X_train).to(device)).cpu().detach().numpy()),
            standartizer.inverse_transform(y_train), squared = False)
            
        wandb.log({"train loss": np.mean(train_loss),
                   "train rmse": np.mean(train_rmse_real)})

        # validation
        val_loss = []
        val_rmse_real = []
        with torch.no_grad(): # не тратим вычислительную мощность
            for x_val, y_val in tqdm(test_loader):
                y_pred = model(x_val)
                loss = criterion(y_pred, y_val)

                val_loss.append(loss.item())
                
        val_rmse_real = mean_squared_error(
            standartizer.inverse_transform(model(torch.FloatTensor(X_test).to(device)).cpu().detach().numpy()),
            standartizer.inverse_transform(y_test), squared = False)
    
        if val_rmse_real < var_rmse_real_min:
          torch.save(model, best_model_path)
          var_rmse_real_min = val_rmse_real

            
        wandb.log({"val loss": np.mean(val_loss),
                   "val rmse": np.mean(val_rmse_real)})    
           
        print(f"Epoch: {epoch}," +
              f"train loss mean: {np.mean(train_loss)}, " +
              f"train rmse: {train_rmse_real}, " +
              f"val loss mean: {np.mean(val_loss)}, " + 
              f"val rmse: {val_rmse_real}")

    wandb.finish()

def test(model, criterion, test_loader):
    '''
    params:
        model - torch.nn.Module to be evaluated on test set
        criterion - loss function from torch.nn
        test_loader - torch.utils.data.Dataloader with test set
    ----------
    returns:
        predicts - torch.tensor with shape (len(test_loader.dataset), ),
                   which contains predictions for test objects
    '''
    predicts = []
    with torch.no_grad():
        for x_val, y_val in test_loader:
            y_pred = model(x_val)
            predicts.extend(y_pred)
    return torch.stack(predicts, dim = 0)

In [ ]:
model_rez_shape = test(model, criterion, test_loader).shape[0]
print(model_rez_shape, y_test.shape[0])
assert model_rez_shape == y_test.shape[0]

In [ ]:
train(model, optimizer, criterion, train_loader, test_loader)

## Задание 2. (0 баллов, но при невыполнении максимум за все задание &mdash; 0 баллов)

Напишите небольшой отчет о том, как вы добились полученного качества: какие средства использовали и какие эксперименты проводили. Подробно расскажите об архитектурах и значениях гиперпараметров, а также какие метрики на тесте они показывали. Чтобы отчет был зачтен, необходимо привести хотя бы 3 эксперимента.

In [ ]:
# лучшее качество
model = torch.load(best_model_path)
model.eval()

mean_squared_error(
            standartizer.inverse_transform(model(torch.FloatTensor(X_test).to(device)).cpu().detach().numpy()),
            standartizer.inverse_transform(y_test), squared = False)

# Report
Какие архитектуры были попробованы:
  0. 90-5000-20000-5000-1000-1 - out of memory error
  1. 90-10000-1000-1, mse loss, adam(lr=0.0003) - test rmse ≥ 9
  2. 90-200-100-1,  mse loss, adam(lr=0.003) - test rmse ≥ 9, при weight decay = 0.01 - test rmse ≥ 8.9
  3. 90-100-256-64-1, mse loss, adam(lr=0.003, weight_decay=0.001) - test rmse ≥ 8.77384175280053
  4. 90-100-256-64-1, mse loss, adam(lr=0.003, weight_decay=0.002) - test rmse ≥ 8.778243071031445
  5. 90-100-256-64-1, mse loss, adam(lr=0.003, weight_decay=0.002) - test rmse ≥ 8.778621852085678
  6. 90-100-256-64-1, RMSE loss, adam(lr=0.003, weight_decay=0.003) - test rmse- rmse ≥ 8.779571473520445
  7. 90-do(0.1)100-do(0.1)256-do(0.1)64-1, RMSE loss, adam(lr=0.0015, weight_decay=0.002) - test rmse ≥ 8.857919035203532
  8. 90-do(0.1)100-do(0.1)256-do(0.1)64-1, RMSE loss, adam(lr=0.003, weight_decay=0.001) - test rmse ≥ 8.82708399272711
  9. 90-do(0.1)100-do(0.1)256-do(0.1)64-1, RMSE loss, adam(lr=0.002, weight_decay=0.001) - test rmse ≥ 8.81
  10. 90-do(0.1)100-do(0.1)256-do(0.1)64-1, RMSE loss, adam(lr=0.0015, weight_decay=0.003) - test rmse ≥ 8.85302287529414
  11. 90-100-256-64-1, RMSE loss, adam(lr=0.003, weight_decay=0.001) + stupid scheduler(2) - rmse ≥ 8.741992526860646 (6th epoch) - победа

Как можно заметить, требуемое качество было достигнуто без применения таких навязанных буржуазным строем излишеств, как nn.Dropout, PCA, batch norm, trorch schedulers.
Таким образом, на практике показана мощь пролетарской четырехслойной архитерктуры с оптимизатором Adam.

Выводы: 
  1. dropout + weight decay = избыточная регуляризация 
  2. расширяться в глубину лучше, чем в ширину
  3. даже, если наша метрика очевидно имеет оптимум в той же точке, что и другая (rmse, mse), лучше обучать модель на той метрике, которую будем мерить 
  4. чтобы долго не ждать 20 эпох обучения, можно переключать lr в процессе оного ( ты управляешь компьютером, а не он тобой )

Красивые графики для привлечения внимания:
https://wandb.ai/lemon-doge/pytorch-hw-1/reports/HW1-graph-report--VmlldzoyNzM5ODU5?accessToken=u62c6pvnhh8yvtl4rfxswbzmj1dcg3zy9wi8lrbzf0a1fs0l6ch66f83mi1g89h2





In [ ]:
# YOUR CODE HERE (－.－)...zzzZZZzzzZZZ
# были архитектуры 
#90-10000-1000-1 adam lr 0.0003 mse loss - rmse ≥ 9
#90-200-100-1 adam lr 0.003 mse loss - rmse ≥ 9, при weight decay rmse ≥ 8.9
#90-100-256-64-1 adam lr 0.003 mse loss weight decay = 0.001 - rmse ≥ 8.77384175280053
#90-100-256-64-1 adam lr 0.003 mse loss weight decay = 0.002 - rmse ≥ 8.778243071031445
#90-100-256-64-1 adam lr 0.0015 RMSE loss weight decay = 0.002 - rmse ≥ 8.778621852085678
#90-100-256-64-1 adam lr 0.0015 RMSE loss weight decay = 0.003 - rmse ≥ 8.779571473520445
#90-do(0.1)100-do(0.1)256-do(0.1)64-1 adam lr 0.0015 RMSE loss weight decay = 0.003 - rmse ≥ 8.857919035203532
#90-do(0.1)100-do(0.1)256-do(0.1)64-1 adam lr 0.0015 RMSE loss weight decay = 0.001 - rmse ≥ 8.82708399272711
#90-do(0.1)100-do(0.1)256-do(0.1)64-1 adam lr 0.002 RMSE loss weight decay = 0.001 - rmse ≥ 8.81
#90-do(0.1)100-do(0.1)256-do(0.1)64-1 adam lr 0.0015 RMSE loss weight decay = 0.003 - rmse ≥ 8.85302287529414
#90-100-256-64-1 adam lr 0.003 mse loss weight decay = 0.001 + stupid scheduler(2) epochs = 10 - rmse ≥ 8.741992526860646 (6th epoch) - победа

Вы можете придерживаться любой адекватной струкуры кода, но мы советуем воспользоваться следующими сигнатурами функций. Лучше всего, если вы проверите ваши предсказания ассертом: так вы убережете себя от разных косяков, например, что вектор предсказаний состоит из всего одного числа. В любом случае, внимательно следите за тем, для каких тензоров вы считаете метрику RMSE. При случайном или намеренном введении в заблуждение проверяющие очень сильно разозлятся.

In [16]:
class CustomDataset(torch.utils.data.Dataset):
    """
    Our custom dataset
    """
    def __init__(self, x, y):
        self.x = torch.FloatTensor(x).to(device)
        self.y = torch.FloatTensor(y).to(device)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx, :], self.y[idx]

In [17]:
import torch.nn.functional as F

In [18]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self,yhat,y):
        return torch.sqrt(self.mse(yhat,y))

In [32]:
train_set = CustomDataset(x = X_train, y = y_train)
train_loader = torch.utils.data.DataLoader(train_set,
                                           batch_size=bs,
                                           shuffle = True)

test_set = CustomDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_set,
                                          batch_size=bs,
                                          shuffle = True)

model = nn.Sequential(
    nn.Linear(X.shape[1], 100), 
    nn.ReLU(),
    nn.Linear(100, 256),
    nn.ReLU(), 
    nn.Linear(256, 64),
    nn.ReLU(), 
    nn.Linear(64, 1),
  ).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay = 0.001)
criterion = RMSELoss()

In [30]:
def train(model, optimizer, criterion, train_loader, test_loader):
    '''
    params:
        model - torch.nn.Module to be fitted
        optimizer - model optimizer
        criterion - loss function from torch.nn
        train_loader - torch.utils.data.Dataloader with train set
        test_loader - torch.utils.data.Dataloader with test set
                      (if you wish to validate during training)
    '''
    wandb.init(project="pytorch-hw-1")
    wandb.watch(model)

    var_rmse_real_min = float("inf")

    for epoch in range(n_epochs):
        # training 
        if(epoch >= 2):
          optimizer.param_groups[0]['lr'] /= 2

        train_loss = []
        train_rmse_real = []
        for x_tr, y_tr in tqdm(train_loader):
            y_pred = model(x_tr)
            loss = criterion(y_pred, y_tr)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            train_loss.append(loss.item())

        train_rmse_real = mean_squared_error(
            standartizer.inverse_transform(model(torch.FloatTensor(X_train).to(device)).cpu().detach().numpy()),
            standartizer.inverse_transform(y_train), squared = False)
            
        wandb.log({"train loss": np.mean(train_loss),
                   "train rmse": np.mean(train_rmse_real)})

        # validation
        val_loss = []
        val_rmse_real = []
        with torch.no_grad(): # не тратим вычислительную мощность
            for x_val, y_val in tqdm(test_loader):
                y_pred = model(x_val)
                loss = criterion(y_pred, y_val)

                val_loss.append(loss.item())
                
        val_rmse_real = mean_squared_error(
            standartizer.inverse_transform(model(torch.FloatTensor(X_test).to(device)).cpu().detach().numpy()),
            standartizer.inverse_transform(y_test), squared = False)
    
        if val_rmse_real < var_rmse_real_min:
          torch.save(model, best_model_path)
          var_rmse_real_min = val_rmse_real

            
        wandb.log({"val loss": np.mean(val_loss),
                   "val rmse": np.mean(val_rmse_real)})    
           
        print(f"Epoch: {epoch}," +
              f"train loss mean: {np.mean(train_loss)}, " +
              f"train rmse: {train_rmse_real}, " +
              f"val loss mean: {np.mean(val_loss)}, " + 
              f"val rmse: {val_rmse_real}")

    wandb.finish()

def test(model, criterion, test_loader):
    '''
    params:
        model - torch.nn.Module to be evaluated on test set
        criterion - loss function from torch.nn
        test_loader - torch.utils.data.Dataloader with test set
    ----------
    returns:
        predicts - torch.tensor with shape (len(test_loader.dataset), ),
                   which contains predictions for test objects
    '''
    predicts = []
    with torch.no_grad():
        for x_val, y_val in test_loader:
            y_pred = model(x_val)
            predicts.extend(y_pred)
    return torch.stack(predicts, dim = 0)

In [21]:
model_rez_shape = test(model, criterion, test_loader).shape[0]
print(model_rez_shape, y_test.shape[0])
assert model_rez_shape == y_test.shape[0]

51630 51630


In [33]:
train(model, optimizer, criterion, train_loader, test_loader)

  0%|          | 0/14492 [00:00<?, ?it/s]

  0%|          | 0/1614 [00:00<?, ?it/s]

Epoch: 0,train loss mean: 0.7956604835200205, train rmse: 8.761703945375217, val loss mean: 0.7938912444582954, val rmse: 8.910302811080218


  0%|          | 0/14492 [00:00<?, ?it/s]

  0%|          | 0/1614 [00:00<?, ?it/s]

...

Epoch: 8,train loss mean: 0.7488892253548968, train rmse: 8.396724098726132, val loss mean: 0.7806087299682837, val rmse: 8.745275116968125


  0%|          | 0/14492 [00:00<?, ?it/s]

  0%|          | 0/1614 [00:00<?, ?it/s]

Epoch: 9,train loss mean: 0.7488843608023447, train rmse: 8.395431640243086, val loss mean: 0.7802100223310228, val rmse: 8.745535875147914


train loss,█▅▄▂▂▁▁▁▁▁
train rmse,█▆▄▂▂▁▁▁▁▁
val loss,█▆▄▃▃▁▁▂▂▂
val rmse,█▆▄▂▂▁▁▁▁▁
train loss,0.74888
train rmse,8.39543
val loss,0.78021
val rmse,8.74554


## Задание 2. (0 баллов, но при невыполнении максимум за все задание &mdash; 0 баллов)

Напишите небольшой отчет о том, как вы добились полученного качества: какие средства использовали и какие эксперименты проводили. Подробно расскажите об архитектурах и значениях гиперпараметров, а также какие метрики на тесте они показывали. Чтобы отчет был зачтен, необходимо привести хотя бы 3 эксперимента.

In [34]:
# лучшее качество
model = torch.load(best_model_path)
model.eval()

mean_squared_error(
            standartizer.inverse_transform(model(torch.FloatTensor(X_test).to(device)).cpu().detach().numpy()),
            standartizer.inverse_transform(y_test), squared = False)

8.741992526860646

# Report
Какие архитектуры были попробованы:
  0. 90-5000-20000-5000-1000-1 - out of memory error
  1. 90-10000-1000-1, mse loss, adam(lr=0.0003) - test rmse ≥ 9
  2. 90-200-100-1,  mse loss, adam(lr=0.003) - test rmse ≥ 9, при weight decay = 0.01 - test rmse ≥ 8.9
  3. 90-100-256-64-1, mse loss, adam(lr=0.003, weight_decay=0.001) - test rmse ≥ 8.77384175280053
  4. 90-100-256-64-1, mse loss, adam(lr=0.003, weight_decay=0.002) - test rmse ≥ 8.778243071031445
  5. 90-100-256-64-1, mse loss, adam(lr=0.003, weight_decay=0.002) - test rmse ≥ 8.778621852085678
  6. 90-100-256-64-1, RMSE loss, adam(lr=0.003, weight_decay=0.003) - test rmse- rmse ≥ 8.779571473520445
  7. 90-do(0.1)100-do(0.1)256-do(0.1)64-1, RMSE loss, adam(lr=0.0015, weight_decay=0.002) - test rmse ≥ 8.857919035203532
  8. 90-do(0.1)100-do(0.1)256-do(0.1)64-1, RMSE loss, adam(lr=0.003, weight_decay=0.001) - test rmse ≥ 8.82708399272711
  9. 90-do(0.1)100-do(0.1)256-do(0.1)64-1, RMSE loss, adam(lr=0.002, weight_decay=0.001) - test rmse ≥ 8.81
  10. 90-do(0.1)100-do(0.1)256-do(0.1)64-1, RMSE loss, adam(lr=0.0015, weight_decay=0.003) - test rmse ≥ 8.85302287529414
  11. 90-100-256-64-1, RMSE loss, adam(lr=0.003, weight_decay=0.001) + stupid scheduler(2) - rmse ≥ 8.741992526860646 (6th epoch) - победа

Как можно заметить, требуемое качество было достигнуто без применения таких навязанных буржуазным строем излишеств, как nn.Dropout, PCA, batch norm, trorch schedulers.
Таким образом, на практике показана мощь пролетарской четырехслойной архитерктуры с оптимизатором Adam.

Выводы: 
  1. dropout + weight decay = избыточная регуляризация 
  2. расширяться в глубину лучше, чем в ширину
  3. даже, если наша метрика очевидно имеет оптимум в той же точке, что и другая (rmse, mse), лучше обучать модель на той метрике, которую будем мерить 
  4. чтобы долго не ждать 20 эпох обучения, можно переключать lr в процессе оного ( ты управляешь компьютером, а не он тобой )

Красивые графики для привлечения внимания:
https://wandb.ai/lemon-doge/pytorch-hw-1/reports/HW1-graph-report--VmlldzoyNzM5ODU5?accessToken=u62c6pvnhh8yvtl4rfxswbzmj1dcg3zy9wi8lrbzf0a1fs0l6ch66f83mi1g89h2





In [23]:
# YOUR CODE HERE (－.－)...zzzZZZzzzZZZ
# были архитектуры 
#90-10000-1000-1 adam lr 0.0003 mse loss - rmse ≥ 9
#90-200-100-1 adam lr 0.003 mse loss - rmse ≥ 9, при weight decay rmse ≥ 8.9
#90-100-256-64-1 adam lr 0.003 mse loss weight decay = 0.001 - rmse ≥ 8.77384175280053
#90-100-256-64-1 adam lr 0.003 mse loss weight decay = 0.002 - rmse ≥ 8.778243071031445
#90-100-256-64-1 adam lr 0.0015 RMSE loss weight decay = 0.002 - rmse ≥ 8.778621852085678
#90-100-256-64-1 adam lr 0.0015 RMSE loss weight decay = 0.003 - rmse ≥ 8.779571473520445
#90-do(0.1)100-do(0.1)256-do(0.1)64-1 adam lr 0.0015 RMSE loss weight decay = 0.003 - rmse ≥ 8.857919035203532
#90-do(0.1)100-do(0.1)256-do(0.1)64-1 adam lr 0.0015 RMSE loss weight decay = 0.001 - rmse ≥ 8.82708399272711
#90-do(0.1)100-do(0.1)256-do(0.1)64-1 adam lr 0.002 RMSE loss weight decay = 0.001 - rmse ≥ 8.81
#90-do(0.1)100-do(0.1)256-do(0.1)64-1 adam lr 0.0015 RMSE loss weight decay = 0.003 - rmse ≥ 8.85302287529414
#90-100-256-64-1 adam lr 0.003 mse loss weight decay = 0.001 + stupid scheduler(2) epochs = 10 - rmse ≥ 8.741992526860646 (6th epoch) - победа